In [ ]:
# Parameters (deze worden overschreven door Papermill)
source = "default_source"
run_ts = "2024-01-01T00:00:00"
process_type = "bronze"

In [ ]:
# Imports
import sys
from pathlib import Path

# Voeg project root toe aan path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.append(str(project_root))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path}")

In [ ]:
# Initialiseer Spark session
from pyspark.sql import SparkSession

# Gebruik getOrCreate om bestaande sessie te hergebruiken
spark = SparkSession.builder \
    .appName(f"Process_{source}_{process_type}") \
    .master("local[2]") \
    .getOrCreate()

print(f"✅ Spark session created: {spark.sparkContext.appName}")
print(f"📊 Spark version: {spark.version}")

In [ ]:
# Toon parameters
print("=" * 70)
print("NOTEBOOK PARAMETERS")
print("=" * 70)
print(f"Source: {source}")
print(f"Run timestamp: {run_ts}")
print(f"Process type: {process_type}")
print("=" * 70)

In [ ]:
# Spark data processing
from pyspark.sql import functions as F
from datetime import datetime

print(f"\n🔄 Processing {source} data with Spark...")
print(f"📅 Timestamp: {run_ts}")
print(f"⚙️  Type: {process_type}")

# Maak sample data
data = [
    (1, "Product A", 100, run_ts),
    (2, "Product B", 200, run_ts),
    (3, "Product C", 150, run_ts),
]

df = spark.createDataFrame(data, ["id", "product", "amount", "timestamp"])

# Voeg derived column toe
df = df.withColumn("category", 
    F.when(F.col("amount") > 150, "high")
     .when(F.col("amount") > 100, "medium")
     .otherwise("low")
)

print(f"\n📊 Data processed:")
df.show()

record_count = df.count()
print(f"✅ Processing complete! Records: {record_count}")

In [ ]:
# Cleanup
spark.stop()
print("🛑 Spark session stopped")